# SI Opportunity Scoring — Train/Validation Comparison (v5)
**Order corrected: Cleaning & filtering first (no numeric mapping inside cleaning)**  
**Fixed rule → Data-driven rule → ML (Calibrated Logistic Regression)**  
**Last updated:** 2026-02-24

## Target (proxy) definition
We derive **`si_offering`** from `OFFERING_NAME`:
- `si_offering_row = 1` if OFFERING_NAME contains token **SI** (case-insensitive)
- `si_offering (per ID) = max(si_offering_row)` across rows for that client

## Objective
Rank **IDs with `si_offering = 0`** by predicted SI alignment from preferences.

## Key design choice
**Cleaning is only cleaning and filtering** (no numeric mapping).
Numeric mappings happen later in **Feature encoding**.

---
## 0) Setup

In [ ]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss

import matplotlib.pyplot as plt
pd.set_option("display.max_columns", 200)

---
## 1) Load raw data + shuffle rows

In [ ]:
DATA_PATH = Path("data.csv")  # <-- change to your real file path

def make_synthetic_data(n=9000, seed=42):
    rng = np.random.default_rng(seed)
    return pd.DataFrame({
        "ID": rng.integers(1, n//2 + 1, size=n),
        "IO_TYPE": rng.choice(["normal", "zombie"], size=n, p=[0.97, 0.03]),
        "LIFE_CYCLE": rng.choice(["open", "closed"], size=n, p=[0.9, 0.1]),
        "OFFERING_NAME": rng.choice(
            ["Core", "Standard", "ESG Plus", "SI Focus", "Core SI", "Income", "SI Sustainable", None],
            size=n, p=[0.23,0.23,0.14,0.14,0.08,0.07,0.06,0.05]
        ),
        "SI_CONSIDERATION_CD": rng.choice(["S1","S2","S3", None], size=n, p=[0.35,0.35,0.2,0.1]),
        "SFDR_PREF": rng.choice(["F1","F2","F3", None], size=n, p=[0.4,0.35,0.2,0.05]),
        "SFDR_ACTUAL": rng.choice(["F1","F2","F3", None], size=n, p=[0.45,0.35,0.15,0.05]),
        "PAI_PREF": rng.choice(["PAI Selected", None], size=n, p=[0.3,0.7]),
        "MIFID": rng.choice(["Yes","No", None], size=n, p=[0.55,0.4,0.05]),
        "TAXONOMYPREF": rng.choice(["A1","A2","A3", None], size=n, p=[0.5,0.35,0.1,0.05]),
        "GHG": rng.choice(["Yes","No","--", None], size=n, p=[0.25,0.65,0.05,0.05]),
        "Biodiversity": rng.choice(["Yes","No","--", None], size=n, p=[0.2,0.7,0.05,0.05]),
        "Water": rng.choice(["Yes","No","--", None], size=n, p=[0.22,0.68,0.05,0.05]),
        "Waste": rng.choice(["Yes","No","--", None], size=n, p=[0.18,0.72,0.05,0.05]),
        "Social": rng.choice(["Yes","No","--", None], size=n, p=[0.28,0.62,0.05,0.05]),
    })

if DATA_PATH.exists():
    df_raw = pd.read_csv(DATA_PATH)
    print(f"Loaded: {DATA_PATH}  shape={df_raw.shape}")
else:
    df_raw = make_synthetic_data()
    print("DATA_PATH not found; using synthetic demo dataset.")
    print(f"shape={df_raw.shape}")

df_raw = df_raw.sample(frac=1, random_state=42).reset_index(drop=True)
df_raw.head()

---
## 2) Cleaning & filtering (ONLY)

Filters and standardization only. No numeric mappings here.

In [ ]:
REQUIRED_RAW = [
    "ID","IO_TYPE","LIFE_CYCLE","OFFERING_NAME",
    "SI_CONSIDERATION_CD","SFDR_PREF","SFDR_ACTUAL","PAI_PREF","MIFID","TAXONOMYPREF",
    "GHG","Biodiversity","Water","Waste","Social"
]
missing_cols = [c for c in REQUIRED_RAW if c not in df_raw.columns]
if missing_cols:
    raise ValueError(f"Missing required raw columns: {missing_cols}")

def clean_filter_only(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # strip whitespace in object cols
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = df[c].apply(lambda x: x.strip() if isinstance(x, str) else x)

    # standardize placeholder missing
    df = df.replace({"--": np.nan, "": np.nan})

    before = len(df)
    df = df[df["IO_TYPE"].fillna("").str.lower() != "zombie"]
    after_zombie = len(df)
    df = df[df["LIFE_CYCLE"].fillna("").str.lower() == "open"]
    after_open = len(df)

    df.attrs["cleaning_summary"] = {
        "before": before,
        "after_remove_zombie": after_zombie,
        "after_keep_open": after_open,
        "removed_zombie": before - after_zombie,
        "removed_closed": after_zombie - after_open
    }
    return df

df_clean_rows = clean_filter_only(df_raw)
pd.DataFrame([df_clean_rows.attrs["cleaning_summary"]])

In [ ]:
s = df_clean_rows.attrs["cleaning_summary"]
plt.figure(figsize=(8,4))
plt.bar(["Before", "After zombie", "After open"], [s["before"], s["after_remove_zombie"], s["after_keep_open"]])
plt.title("Cleaning impact: rows remaining after filters")
plt.ylabel("Rows")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

key_cols = ["SI_CONSIDERATION_CD","SFDR_PREF","SFDR_ACTUAL","TAXONOMYPREF","MIFID","PAI_PREF",
            "GHG","Biodiversity","Water","Waste","Social","OFFERING_NAME"]
miss = df_clean_rows[key_cols].isna().mean().sort_values(ascending=False)

display(miss.to_frame("missing_rate").head(12))

plt.figure(figsize=(10,4))
plt.bar(miss.index[:12], miss.values[:12])
plt.title("Top missing rates after cleaning (raw values)")
plt.ylabel("Missing rate")
plt.xticks(rotation=45, ha="right")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

---
## 3) Derive SI label + aggregate to ID-level

In [ ]:
df = df_clean_rows.copy()
df["si_offering_row"] = df["OFFERING_NAME"].astype(str).str.contains(r"\bSI\b", case=False, na=False).astype(int)

def mode_or_first(s: pd.Series):
    s2 = s.dropna()
    if len(s2) == 0:
        return np.nan
    m = s2.mode()
    return m.iloc[0] if len(m) else s2.iloc[0]

agg_dict = {
    "IO_TYPE": mode_or_first,
    "LIFE_CYCLE": mode_or_first,
    "OFFERING_NAME": mode_or_first,
    "SI_CONSIDERATION_CD": mode_or_first,
    "SFDR_PREF": mode_or_first,
    "SFDR_ACTUAL": mode_or_first,
    "PAI_PREF": mode_or_first,
    "MIFID": mode_or_first,
    "TAXONOMYPREF": mode_or_first,
    "GHG": mode_or_first,
    "Biodiversity": mode_or_first,
    "Water": mode_or_first,
    "Waste": mode_or_first,
    "Social": mode_or_first,
    "si_offering_row": "max",
}

df_id = df.groupby("ID", as_index=False).agg(agg_dict).rename(columns={"si_offering_row":"si_offering"})

sizes = pd.DataFrame({
    "level": ["row-level (cleaned)", "ID-level"],
    "rows": [len(df), len(df_id)],
    "si_offering_rate": [df["si_offering_row"].mean(), df_id["si_offering"].mean()]
})
display(sizes)

plt.figure(figsize=(7,4))
plt.bar(sizes["level"], sizes["rows"])
plt.title("Size: cleaned rows vs ID-level")
plt.ylabel("Count")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,4))
plt.bar(sizes["level"], sizes["si_offering_rate"])
plt.title("Proxy label prevalence: si_offering rate")
plt.ylabel("Rate")
plt.ylim(0, 1)
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

df_id.head()

---
## 4) Feature encoding and engineering (numbers happen here)

In [ ]:
MAP_SI = {"S1":1, "S2":2, "S3":3}
MAP_SFDR = {"F1":1, "F2":2, "F3":3}
MAP_TAX = {"A1":1, "A2":2, "A3":3}

def yes_to_1(x):
    return 1 if isinstance(x, str) and x.strip().lower() == "yes" else 0

def encode_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    for c in ["GHG","Biodiversity","Water","Waste","Social"]:
        df[c] = df[c].apply(yes_to_1).astype(int)

    df["MIFID"] = df["MIFID"].apply(yes_to_1).astype(int)
    df["PAI_PREF"] = (df["PAI_PREF"].astype(str).str.lower() == "pai selected").astype(int)

    df["SI_CONSIDERATION_num"] = df["SI_CONSIDERATION_CD"].map(MAP_SI).fillna(1).astype(int)
    df["SFDR_PREF_num"] = df["SFDR_PREF"].map(MAP_SFDR).fillna(1).astype(int)
    df["SFDR_ACTUAL_num"] = df["SFDR_ACTUAL"].map(MAP_SFDR).fillna(1).astype(int)
    df["TAXONOMYPREF_num"] = df["TAXONOMYPREF"].map(MAP_TAX).fillna(1).astype(int)

    df["sfdr_gap"] = np.clip(df["SFDR_PREF_num"] - df["SFDR_ACTUAL_num"], -2, 2)
    df["sfdr_opp"] = np.maximum(df["sfdr_gap"], 0)

    topic_cols = ["GHG","Biodiversity","Water","Waste","Social"]
    df["esg_topics_yes_cnt"] = df[topic_cols].sum(axis=1)
    df["esg_topics_yes_share"] = df["esg_topics_yes_cnt"] / len(topic_cols)

    df["si_norm"] = np.clip((df["SI_CONSIDERATION_num"] - 1)/2, 0, 1)
    df["sfdr_norm"] = np.clip(df["sfdr_opp"]/2, 0, 1)
    df["topics_norm"] = np.clip(df["esg_topics_yes_share"], 0, 1)
    df["topics_if_pai"] = df["topics_norm"] * df["PAI_PREF"]
    df["tax_norm"] = np.clip((df["TAXONOMYPREF_num"] - 1)/2, 0, 1)

    df["si_offering"] = df["si_offering"].astype(int)
    return df

df_feat = encode_features(df_id)

plt.figure(figsize=(7,4))
plt.hist(df_feat["sfdr_gap"], bins=5)
plt.title("Distribution of sfdr_gap")
plt.xlabel("sfdr_gap")
plt.ylabel("count")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,4))
plt.hist(df_feat["TAXONOMYPREF_num"], bins=3)
plt.title("Distribution of TAXONOMYPREF_num (A1/A2/A3)")
plt.xlabel("TAXONOMYPREF_num")
plt.ylabel("count")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

df_feat[["ID","si_offering","MIFID","sfdr_gap","PAI_PREF","TAXONOMYPREF_num","esg_topics_yes_cnt"]].head()

---
## 5) Fixed-weight rule score (branching) — includes Taxonomy

In [ ]:
@dataclass
class RuleConfig:
    si_score_s1: float = 20
    si_score_s2: float = 50
    si_score_s3: float = 80
    w_sfdr: float = 0.60
    w_pai_block: float = 0.25
    w_tax: float = 0.15

cfg = RuleConfig()

def score_fixed_rule(df: pd.DataFrame, cfg: RuleConfig) -> pd.Series:
    si_score = df["SI_CONSIDERATION_num"].map({1: cfg.si_score_s1, 2: cfg.si_score_s2, 3: cfg.si_score_s3}).astype(float)
    pai_block = np.where(df["PAI_PREF"] == 1, 0.5 + 0.5*df["topics_norm"], 0.0)
    score_B = 100 * (cfg.w_sfdr*df["sfdr_norm"] + cfg.w_pai_block*pai_block + cfg.w_tax*df["tax_norm"])
    score = np.where(df["MIFID"]==1, score_B, si_score)
    return pd.Series(np.clip(score, 0, 100), index=df.index)

df_feat["score_fixed"] = score_fixed_rule(df_feat, cfg)

plt.figure(figsize=(7,4))
plt.hist(df_feat["score_fixed"], bins=30)
plt.title("Score distribution: Fixed-weight rule")
plt.xlabel("score_fixed")
plt.ylabel("count")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 6) Train/Validation split (ID-level)

In [ ]:
FEATURES_ALL = [
    "si_norm","sfdr_norm","PAI_PREF","topics_if_pai","tax_norm",
    "esg_topics_yes_cnt","sfdr_gap","MIFID","SI_CONSIDERATION_num","TAXONOMYPREF_num"
]

X = df_feat[FEATURES_ALL].copy()
y = df_feat["si_offering"].copy()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)
idx_train, idx_val = X_train.index, X_val.index

print("Train:", len(idx_train), "Val:", len(idx_val))
print("Train si rate:", y_train.mean().round(4), "Val si rate:", y_val.mean().round(4))

---
## 7) Evaluation helpers

In [ ]:
def eval_scores(y_true, p, label):
    return {"model": label,
            "auc": roc_auc_score(y_true, p),
            "avg_precision": average_precision_score(y_true, p),
            "brier": brier_score_loss(y_true, p)}

def lift_table(y_true, p, n_bins=10):
    tmp = pd.DataFrame({"y": y_true, "p": p})
    tmp["bin"] = pd.qcut(tmp["p"], n_bins, labels=False, duplicates="drop") + 1
    return tmp.groupby("bin")["y"].agg(["mean","count"]).rename(columns={"mean":"si_rate"})

def plot_lift(tab, title):
    plt.figure(figsize=(8,4))
    plt.plot(tab.index, tab["si_rate"].values, marker="o")
    plt.title(title)
    plt.xlabel("Decile (1=lowest, 10=highest)")
    plt.ylabel("si_offering rate")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

---
## 8) Method 1 — Fixed-weight rule (validation)

In [ ]:
p_fixed_val = df_feat.loc[idx_val, "score_fixed"].values / 100.0
fixed_metrics = eval_scores(y_val.values, p_fixed_val, "Fixed-weight rule")
display(fixed_metrics)

lift_fixed = lift_table(y_val.values, p_fixed_val)
plot_lift(lift_fixed, "Lift (proxy): Fixed-weight rule")
display(lift_fixed)

prob_true, prob_pred = calibration_curve(y_val.values, p_fixed_val, n_bins=10, strategy="quantile")
plt.figure(figsize=(6,6))
plt.plot(prob_pred, prob_true, marker="o")
plt.plot([0,1],[0,1], linestyle="--")
plt.title("Calibration (proxy): Fixed-weight rule")
plt.xlabel("Predicted (score_fixed/100)")
plt.ylabel("Observed si rate")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 9) Method 2 — Data-driven rule (learn Branch B weights on train)

In [ ]:
train_B = df_feat.loc[idx_train]
train_B = train_B[train_B["MIFID"]==1].copy()

DD_B_FEATURES = ["sfdr_norm","PAI_PREF","topics_if_pai","tax_norm"]
lr_dd_B = LogisticRegression(max_iter=6000, class_weight="balanced")
lr_dd_B.fit(train_B[DD_B_FEATURES], train_B["si_offering"])

coef_B = pd.Series(lr_dd_B.coef_[0], index=DD_B_FEATURES).sort_values(key=np.abs, ascending=False)
display(coef_B.to_frame("coef"))

pos = np.maximum(coef_B.values, 0)
if pos.sum() == 0:
    pos = np.ones_like(pos)
w_dd_B = pd.Series(100*pos/pos.sum(), index=coef_B.index).sort_values(ascending=False)
display(w_dd_B.to_frame("weight_100"))

plt.figure(figsize=(7,4))
plt.bar(w_dd_B.index, w_dd_B.values)
plt.title("Learned Branch-B weights (sum=100)")
plt.ylabel("weight")
plt.xticks(rotation=45, ha="right")
plt.grid(True, axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

# Score
df_feat["score_datadriven"] = np.nan
si_score = df_feat["SI_CONSIDERATION_num"].map({1: cfg.si_score_s1, 2: cfg.si_score_s2, 3: cfg.si_score_s3}).astype(float)
df_feat.loc[df_feat["MIFID"]==0, "score_datadriven"] = si_score[df_feat["MIFID"]==0]

scoreB = (w_dd_B["sfdr_norm"]*df_feat["sfdr_norm"] +
          w_dd_B["PAI_PREF"]*df_feat["PAI_PREF"] +
          w_dd_B["topics_if_pai"]*df_feat["topics_if_pai"] +
          w_dd_B["tax_norm"]*df_feat["tax_norm"])
df_feat.loc[df_feat["MIFID"]==1, "score_datadriven"] = scoreB[df_feat["MIFID"]==1]
df_feat["score_datadriven"] = df_feat["score_datadriven"].clip(0,100)

p_dd_val = df_feat.loc[idx_val, "score_datadriven"].values / 100.0
dd_metrics = eval_scores(y_val.values, p_dd_val, "Data-driven rule")
display(dd_metrics)

lift_dd = lift_table(y_val.values, p_dd_val)
plot_lift(lift_dd, "Lift (proxy): Data-driven rule")
display(lift_dd)

prob_true, prob_pred = calibration_curve(y_val.values, p_dd_val, n_bins=10, strategy="quantile")
plt.figure(figsize=(6,6))
plt.plot(prob_pred, prob_true, marker="o")
plt.plot([0,1],[0,1], linestyle="--")
plt.title("Calibration (proxy): Data-driven rule")
plt.xlabel("Predicted (score_datadriven/100)")
plt.ylabel("Observed si rate")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 10) Method 3 — ML: Calibrated Logistic Regression (train → validate)

In [ ]:
ML_FEATURES = FEATURES_ALL
Xtr = X_train[ML_FEATURES].copy()
Xva = X_val[ML_FEATURES].copy()

lr = LogisticRegression(max_iter=7000, class_weight="balanced")
cal_lr = CalibratedClassifierCV(lr, method="isotonic", cv=5)
cal_lr.fit(Xtr, y_train)

p_lr_val = cal_lr.predict_proba(Xva)[:,1]
ml_metrics = eval_scores(y_val.values, p_lr_val, "ML: Calibrated Logistic Regression")
display(ml_metrics)

lift_lr = lift_table(y_val.values, p_lr_val)
plot_lift(lift_lr, "Lift (proxy): ML Calibrated Logistic Regression")
display(lift_lr)

prob_true, prob_pred = calibration_curve(y_val.values, p_lr_val, n_bins=10, strategy="quantile")
plt.figure(figsize=(6,6))
plt.plot(prob_pred, prob_true, marker="o")
plt.plot([0,1],[0,1], linestyle="--")
plt.title("Calibration (proxy): ML Calibrated Logistic Regression")
plt.xlabel("Predicted probability")
plt.ylabel("Observed si rate")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---
## 11) Compare validation performance (3 methods)

In [ ]:
comparison = pd.DataFrame([fixed_metrics, dd_metrics, ml_metrics]).round(4)
display(comparison.sort_values("auc", ascending=False))

for metric in ["auc","avg_precision","brier"]:
    plt.figure(figsize=(8,4))
    plt.bar(comparison["model"], comparison[metric])
    plt.title(f"Validation comparison: {metric}")
    plt.ylabel(metric)
    plt.xticks(rotation=30, ha="right")
    plt.grid(True, axis="y", alpha=0.3)
    plt.tight_layout()
    plt.show()

---
## 12) Operational output: rank `si_offering=0` IDs + buckets

In [ ]:
RANK_METHOD = "score_datadriven"  # score_fixed or score_datadriven

df_out = df_feat.copy()
df_out["rank_score"] = df_out[RANK_METHOD]
df_out["score_percentile"] = (df_out["rank_score"].rank(pct=True) * 100).round(2)
df_out["bucket_3"] = pd.cut(df_out["score_percentile"], bins=[-0.01, 50, 80, 100], labels=["Low","Average","High"])

targets = df_out[df_out["si_offering"]==0].sort_values("rank_score", ascending=False)
targets[["ID","rank_score","score_percentile","bucket_3","MIFID","sfdr_gap","PAI_PREF","TAXONOMYPREF_num","esg_topics_yes_cnt"]].head(20)

---
## 13) Pilot plan

Use High bucket for treatment, random control, measure response/adoption, retrain on true outcomes.